# Parameters
in this cell the used variables can be changed 

height and width correspond to the dimensions of the Workspace, in which the Topology Optimisation is going to be performed

log_dir is the direction where the current best Model will be saved to during training

ts is the amount of timesteps the model will be trained for




In [20]:
import numpy as np
import gymnasium as gym

import autograd.numpy as anp

from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter

In [2]:
height = 6
width = 20
log_dir = "log"
ts = 1e6

density = 1e-4


normals = np.zeros((height, width, 2))
x = 0
y = 1
print("normalsX\n" + str(normals[:,:,x]))
print("normalsY\n" + str(normals[:,:,y]))



number_of_nodes = height * width

normalsX
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
normalsY
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
class ObjectView(object):
    def __init__(self, d): self.__dict__ = d
    
def get_args(normals, forces, density=1e-4):  # Manage the problem setup parameters
    width = normals.shape[0] - 1
    height = normals.shape[1] - 1
    fixdofs = np.flatnonzero(normals.ravel())
    alldofs = np.arange(2 * (width + 1) * (height + 1))
    freedofs = np.sort(list(set(alldofs) - set(fixdofs)))
    params = {
      # material properties
      'young': 1, 'young_min': 1e-9, 'poisson': 0.3, 'g': 0,
      # constraints
      'density': density, 'xmin': 0.001, 'xmax': 1.0,
      # input parameters
      'nelx': width, 'nely': height, 'mask': 1, 'penal': 3.0, 'filter_width': 1,
      'freedofs': freedofs, 'fixdofs': fixdofs, 'forces': forces.ravel(),
      # optimization parameters
      'opt_steps': 80, 'print_every': 10}
    return ObjectView(params)

def mbb_beam(width=7, height=7, density=1e-4, y=1, x=0, rd=-1):  # textbook beam example
    normals = np.zeros((width + 1, height + 1, 2))
    normals[0, 0, x] = 1
    normals[0, 0, y] = 1
    normals[0, -1, x] = 1
    normals[0, -1, y] = 1
    forces = np.zeros((width + 1, height + 1, 2))
    forces[-1, rd, y] = -1
    return normals, forces, density

In [41]:
class TopOptEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self):
        super().__init__()
        self.workspace = anp.ones((height, width)) * density

    
        self.action_space = gym.spaces.Discrete(number_of_nodes)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(height, width, 3), dtype=np.float64)

        self.n, self.m = self.workspace.shape
        print("self.n:\t" + str(self.n) + "\nself.m:\t" + str(self.m))
        self._actions_to_coordinates={} 
    
        k=0
        for i in range(self.n):
            for j in range(self.m):
                self._actions_to_coordinates[k]=(i,j)
                k+=1
        print(self._actions_to_coordinates)
        self.reward = 0
        self.step_count = 0

    def step(self, action):
        self.step_count += 1
        

        self.tmp, self.const = fast_stopt(self.args, self.x)

        self.reward = rewardFunc(self.workspace)
        return self.workspace, self.reward, self.step, {}
    

    def reset(self, seed=None):
        super().reset(seed=seed)

        self.workspace = anp.ones((height, width)) * density
        print(self.workspace.reshape(self.workspace.shape[0] * self.workspace.shape[1]))

        # Flattens the array from a 2D array into a one dimensional array for the observation space

        return self.workspace.reshape(self.workspace.shape[0] * self.workspace.shape[1]), self._getInfo()
    
    def _getInfo(self):
        return {"number of steps": self.step_count, 
                "current Reward": self.reward,
                "Workspace": self.workspace}


    

        


In [42]:
env = TopOptEnv()
print(env._getInfo())

self.n:	6
self.m:	20
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (0, 4), 5: (0, 5), 6: (0, 6), 7: (0, 7), 8: (0, 8), 9: (0, 9), 10: (0, 10), 11: (0, 11), 12: (0, 12), 13: (0, 13), 14: (0, 14), 15: (0, 15), 16: (0, 16), 17: (0, 17), 18: (0, 18), 19: (0, 19), 20: (1, 0), 21: (1, 1), 22: (1, 2), 23: (1, 3), 24: (1, 4), 25: (1, 5), 26: (1, 6), 27: (1, 7), 28: (1, 8), 29: (1, 9), 30: (1, 10), 31: (1, 11), 32: (1, 12), 33: (1, 13), 34: (1, 14), 35: (1, 15), 36: (1, 16), 37: (1, 17), 38: (1, 18), 39: (1, 19), 40: (2, 0), 41: (2, 1), 42: (2, 2), 43: (2, 3), 44: (2, 4), 45: (2, 5), 46: (2, 6), 47: (2, 7), 48: (2, 8), 49: (2, 9), 50: (2, 10), 51: (2, 11), 52: (2, 12), 53: (2, 13), 54: (2, 14), 55: (2, 15), 56: (2, 16), 57: (2, 17), 58: (2, 18), 59: (2, 19), 60: (3, 0), 61: (3, 1), 62: (3, 2), 63: (3, 3), 64: (3, 4), 65: (3, 5), 66: (3, 6), 67: (3, 7), 68: (3, 8), 69: (3, 9), 70: (3, 10), 71: (3, 11), 72: (3, 12), 73: (3, 13), 74: (3, 14), 75: (3, 15), 76: (3, 16), 77: (3, 17), 78: (3, 18), 7